In [1]:
from src.ga_api_package.ga_api_client import Repository, HyperLogLog
import pandas as pd
import logging

In [2]:
logging.basicConfig(
    format='▸ %(asctime)s %(levelname)s %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.DEBUG
)

In [3]:
repo = Repository('https://rdlab-214.genie-analytics.com/api', 'api', 'default', 'api123!@#', burst=1)

In [4]:
pipeline = {
    'scope': {
        'i-field': '@controller.home',
        'i-entry': [
            [True,'t:0:0:1',0]
        ],
        'mode': '%directional'
    },
    'bucket': [
        ['$distinctTuple', {
            'fields':['@flow.addr.src','@flow.port.src','@flow.addr.dst'],
            'unwind':[False,False],
            'sizeLimit':1_000_000,
            'sortedBy':['$sum', {
                'field':'@flow.bytes'
            }],
            'sortedDir':1,
            'nullOpt':False
        }]
    ],
    'metric': [
        ['$sum', {
            'field':'@flow.bytes'
        }]
    ]
}

In [5]:
freq = 1800
delta = pd.Timedelta(f'{freq}S')
adhoc = await repo.set_adhoc(freq, pipeline=pipeline)
start = pd.Timestamp.now('+08:00').floor(delta) - delta
dts = pd.date_range(start=start, periods=1, freq=f'{freq}S')
df = await adhoc.read_data(dts, 'full')
df

▸ 2022-03-17 18:20:27 INFO ga_api_client.py:_authenticate:132 - bringup
▸ 2022-03-17 18:20:27 INFO ga_api_client.py:_authenticate:163 - authenticate
▸ 2022-03-17 18:20:32 INFO ga_api_client.py:create_adhoc:350 - status=201 rlt=212233239
▸ 2022-03-17 18:20:32 INFO ga_api_client.py:create_adhoc:350 - status=201 rlt=212233239
▸ 2022-03-17 18:20:32 ERROR ga_api_client.py:_request:208 - status=500 message={"error":"Assign worker failed - memory full"}


""


In [6]:
await repo.close()